In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
data = pd.read_csv('/Users/jonathantao/Downloads/filename.csv', encoding='utf-8') #or 'windows-1252' if utf-8 errors
data = data.drop_duplicates(keep='first', subset=['pubmedurls_Article_PMID'])
data.head()


In [ ]:
len(data)

In [ ]:
from pymed import PubMed
from bs4 import BeautifulSoup
import requests
import re
import math

pubmed = PubMed(tool="macOS VSCode", email="beneric1232003@gmail.com")

In [ ]:
import csv 
with open('output_file_namae.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Indicator", "MoA", "Title", "URL", "Author", "PMID", "Abstract", "Trial ID", "Phase", "Date"])
    i = 0
    currMoa = "None"
    for index, row in data.iterrows(): 
        print(i)
        #PMID 
        ID = str(row['pubmedurls_Article_PMID'])
        
        #Abstract 
        try:
            article = pubmed.query(ID)
            article_content = next(article)
            abstract = article_content.abstract
            title = article_content.title
        except:
            abstract = "None"
            title = "None"
            print("Could not find abstract for: " + ID)
        
        #Mesh Terms and URL
        terms_list = []
        try:
            url = row["pubmedurls_Article_URL"]
            pmid = ID
            req = requests.get(url)
            soup = BeautifulSoup(req.content, "html.parser")
            
            mesh_section = soup.find("div", attrs={'id': 'mesh-terms', 'class': 'mesh-terms keywords-section'})
            keywords_list = mesh_section.find("ul", attrs={'class': 'keywords-list'})
            li_list = keywords_list.find_all("li")
            for li_indiv in li_list:
                try:
                    text_div = li_indiv.find("div", attrs={'class': 'keyword-actions dropdown-block'})
                    button_div = text_div.find("button")
                    #print(button_div.text)
                    button_text_raw = button_div.text
                    button_text_cleaned = button_text_raw.strip()
                    terms_list.append(button_text_cleaned)
                except:
                    pass
        except:
            print("Could not find mesh terms for: " + ID)
        abstract+=' '.join(terms_list)
    
        #MoA
        moa_list = ["partial", "antagonist", "inverse", "inhibitor", "activator", "stimulant", "modulator", "replacement", "antimetabolite", "blocker", "inducer", "agonist"]
        """
        if isinstance(row["MoA"], str):
            moa = row["MoA"]
            currMoa = moa
        else: 
            moa = currMoa
        """
        raw_search_url = row["pubmedurls_link"].lower()
        actual_moa = "None"
        for moa in moa_list:
                if moa in raw_search_url:
                    if moa == "partial":
                        actual_moa = "partial agonist"
                    elif moa == "inverse":
                        actual_moa = "inverse agonist"
                    else:
                        actual_moa = moa
                    break
        
        #Title and Authors
        try:
            title = row["pubmedurls_Article_Title"]
            authors = row["pubmedurls_Article_Authors"]
        except:
            title = "None"
            authors = "None"
        
        #date 
        try:
            raw_data = row["pubmedurls_Article_Misc"]
            dates_regex = "(\d\d\d\d.*?);"
            dates = re.findall(f"{dates_regex}", raw_data)[0]
        except:
            dates = "None"
        
        #phase
        try:
            req = requests.get(url)
            soup = BeautifulSoup(req.content, "html.parser")
            link = soup.find("a", attrs={'title': 'See in ClinicalTrials.gov'})
            raw_text = link.text
            trial_id = raw_text.strip()
            URL = "https://clinicaltrials.gov/ct2/show/" + trial_id
            try: 
                page = requests.get(URL)
                soup = BeautifulSoup(page.content, "html.parser")
                table_contents = soup.find_all("span", attrs={'style': 'display:block;margin-bottom:1ex;'})
                phase = table_contents[-1].text
            except:
                phase = "Not Found"
        except:
            trial_id = "None"
            phase = "Not Found"
        if phase.find("Phase") == -1:
            phase = "Not Found"     

        
        #write
        writer.writerow(["disease name", actual_moa, title, url, authors, pmid, abstract, trial_id, phase, dates])
        i+=1